In [ ]:
!pip install nibabel
!pip install pyradiomics
!pip install pynrrd
!pip install SimpleITK

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import nrrd
from scipy import ndimage
import matplotlib.pyplot as plt
import os
import radiomics
from radiomics import featureextractor
import logging
import SimpleITK as sitk

radiomics.logger.setLevel(logging.ERROR)

extractor = featureextractor.RadiomicsFeatureExtractor('/content/drive/MyDrive/UCSF-GBM/PyRadiomics/exampleMR_3mm.yaml')

In [ ]:
def binary_mask(mask_path, patient):
    mask = nib.load(mask_path)
    mask = mask.get_fdata()
    #Convert mask to binary.
    mask[mask == 2] = -1
    binary_mask = mask > 0
    mask = nib.load(mask_path)
    binary_mask = nib.Nifti1Image(binary_mask, mask.affine, mask.header)
    #Save binary mask.
    path, filename = os.path.split(mask_path)
    filename = path + '/' + patient + '_binary_mask' + '.nii'
    nib.save(binary_mask, str(filename))
    return

In [ ]:
# Set the file paths
sample_path = '/content/drive/MyDrive/HCMP-ML/MRI/NO VT/HCMP_003/HCMP_003_Cine.nii'
sample_mask_path = '/content/drive/MyDrive/HCMP-ML/MRI/NO VT/HCMP_003/HCMP_003_Cine_Seg.nii'

# Read the image and mask using SimpleITK
sample_image = sitk.ReadImage(sample_path)
sample_mask = sitk.ReadImage(sample_mask_path)

# Execute the feature extraction
sample = extractor.execute(sample_image, sample_mask)

# Process the results
sample = dict(sample)
sample = {k: v for k, v in sample.items() if not k.startswith('diagnostics')}

In [ ]:
LGE_VT = []
Cine_VT = []

p = '/content/drive/MyDrive/HCMP-ML/MRI/VT'
patients = os.listdir(p)

for patient in patients:

  index = []
  index.append(patient)

  LGE_p = p + '/' + patient + '/' + patient + '_LGE.nii'
  LGEm_p = p + '/' + patient + '/' + patient + '_LGE_Seg.nii'

  # Read the image and mask using SimpleITK
  LGE_image = sitk.ReadImage(LGE_p)
  LGE_mask = sitk.ReadImage(LGEm_p)

  print('Extracting LGE features of: ' + str(patient))
  LGE_features = extractor.execute(LGE_image, LGE_mask)
  LGE_features = dict(LGE_features)
  LGE_features = {k: v for k, v in LGE_features.items() if not k.startswith('diagnostics')}
  LGE_features = pd.DataFrame([LGE_features], index=index)
  LGE_VT.append(LGE_features)
  print('Number of total features extracted from LGE images of VT patients: ' + str(len(LGE_VT)))

  Cine_p = p + '/' + patient + '/' + patient + '_Cine.nii'
  Cinem_p = p + '/' + patient + '/' + patient + '_Cine_Seg.nii'

  # Read the image and mask using SimpleITK
  Cine_image = sitk.ReadImage(Cine_p)
  Cine_mask = sitk.ReadImage(Cinem_p)

  print('Extracting Cine features of: ' + str(patient))
  Cine_features = extractor.execute(Cine_image, Cine_mask)
  Cine_features = dict(Cine_features)
  Cine_features = {k: v for k, v in Cine_features.items() if not k.startswith('diagnostics')}
  Cine_features = pd.DataFrame([Cine_features], index=index)
  Cine_VT.append(Cine_features)
  print('Number of total features extracted from Cine images of VT patients: ' + str(len(Cine_VT)))

In [ ]:
LGE_VT = pd.concat(LGE_VT)
LGE_VT = LGE_VT.add_prefix('LGE_')
Cine_VT = pd.concat(Cine_VT)
Cine_VT = Cine_VT.add_prefix('Cine_')

VT = pd.concat([LGE_VT, Cine_VT], axis=1)
VT = VT.assign(VT=1)
VT.to_csv('/content/drive/MyDrive/HCMP-ML/VT.csv')

LGE_VT = LGE_VT.assign(VT=1)
LGE_VT.to_csv('/content/drive/MyDrive/HCMP-ML/LGE_VT.csv')

Cine_VT = Cine_VT.assign(VT=1)
Cine_VT.to_csv('/content/drive/MyDrive/HCMP-ML/Cine_VT.csv')

In [ ]:
LGE_NO_VT = []
Cine_NO_VT = []

p = '/content/drive/MyDrive/HCMP-ML/MRI/NO_VT'
patients = os.listdir(p)

for patient in patients:

  index = []
  index.append(patient)

  LGE_p = p + '/' + patient + '/' + patient + '_LGE.nii'
  LGEm_p = p + '/' + patient + '/' + patient + '_LGE_Seg.nii'

  # Read the image and mask using SimpleITK
  LGE_image = sitk.ReadImage(LGE_p)
  LGE_mask = sitk.ReadImage(LGEm_p)

  print('Extracting LGE features of: ' + str(patient))
  LGE_features = extractor.execute(LGE_image, LGE_mask)
  LGE_features = dict(LGE_features)
  LGE_features = {k: v for k, v in LGE_features.items() if not k.startswith('diagnostics')}
  LGE_features = pd.DataFrame([LGE_features], index=index)
  LGE_NO_VT.append(LGE_features)
  print('Number of total features extracted from LGE images of NO_VT patients: ' + str(len(LGE_NO_VT)))

  Cine_p = p + '/' + patient + '/' + patient + '_Cine.nii'
  Cinem_p = p + '/' + patient + '/' + patient + '_Cine_Seg.nii'

  # Read the image and mask using SimpleITK
  Cine_image = sitk.ReadImage(Cine_p)
  Cine_mask = sitk.ReadImage(Cinem_p)

  print('Extracting Cine features of: ' + str(patient))
  Cine_features = extractor.execute(Cine_image, Cine_mask)
  Cine_features = dict(Cine_features)
  Cine_features = {k: v for k, v in Cine_features.items() if not k.startswith('diagnostics')}
  Cine_features = pd.DataFrame([Cine_features], index=index)
  Cine_NO_VT.append(Cine_features)
  print('Number of total features extracted from Cine images of NO_VT patients: ' + str(len(Cine_NO_VT)))

In [ ]:
LGE_NO_VT = pd.concat(LGE_NO_VT)
LGE_NO_VT = LGE_NO_VT.add_prefix('LGE_')
Cine_NO_VT = pd.concat(Cine_NO_VT)
Cine_NO_VT = Cine_NO_VT.add_prefix('Cine_')

NO_VT = pd.concat([LGE_NO_VT, Cine_NO_VT], axis=1)
NO_VT = NO_VT.assign(VT=0)
NO_VT.to_csv('/content/drive/MyDrive/HCMP-ML/NO_VT.csv')

LGE_NO_VT = LGE_NO_VT.assign(T=0)
LGE_NO_VT.to_csv('/content/drive/MyDrive/HCMP-ML/LGE_NO_VT.csv')

Cine_NO_VT = Cine_NO_VT.assign(VT=0)
Cine_NO_VT.to_csv('/content/drive/MyDrive/HCMP-ML/Cine_NO_VT.csv')

In [ ]:
#Merge VT and NO_VT dataframes.

LGE = pd.concat([LGE_VT, LGE_NO_VT], axis=0)
LGE = LGE.sort_index(ascending=True)
LGE.to_csv('/content/drive/MyDrive/HCMP-ML/LGE.csv')

Cine = pd.concat([Cine_VT, Cine_NO_VT], axis=0)
Cine = Cine.sort_index(ascending=True)
Cine.to_csv('/content/drive/MyDrive/HCMP-ML/Cine.csv')

Combined = pd.concat([VT, NO_VT], axis=0)
Combined = Combined.sort_index(ascending=True)
Combined.to_csv('/content/drive/MyDrive/HCMP-ML/Combined.csv')

In [ ]:
#Min-Max scale and save the scaled data.

LGE_scaled = (LGE-LGE.min())/(LGE.max()-LGE.min())
LGE_scaled.to_csv('/content/drive/MyDrive/HCMP-ML/LGE_scaled.csv')

Cine_scaled = (Cine-Cine.min())/(Cine.max()-Cine.min())
Cine_scaled.to_csv('/content/drive/MyDrive/HCMP-ML/Cine_scaled.csv')

Combined_scaled = (Combined-Combined.min())/(Combined.max()-Combined.min())
Combined_scaled.to_csv('/content/drive/MyDrive/HCMP-ML/Combined_scaled.csv')